In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer 

/workspaces/hf-nlp/transformers-nlp/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets['train'] = raw_datasets['train'].shuffle(seed=42).select(range(500))
raw_datasets['validation'] = raw_datasets['validation'].shuffle(seed=42).select(range(100))
raw_datasets['test'] = raw_datasets['test'].shuffle(seed=42).select(range(200))

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 200/200 [00:00<00:00, 5627.86 examples/s]


In [3]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
training_args = TrainingArguments("test-trainer",evaluation_strategy="epoch")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/workspaces/hf-nlp/transformers-nlp/.env/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
import numpy as np
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_30199/2871184925.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.609200,0.670000,0.795031
2,No log,0.564606,0.730000,0.813793
3,No log,0.587057,0.750000,0.812030


TrainOutput(global_step=189, training_loss=0.44599906477347884, metrics={'train_runtime': 803.3073, 'train_samples_per_second': 1.867, 'train_steps_per_second': 0.235, 'total_flos': 54914155030800.0, 'train_loss': 0.44599906477347884, 'epoch': 3.0})